In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import hf

In [2]:
def ground_state_VQE(H):
    """Perform VQE to find the ground state of the H2 Hamiltonian.
    Args:
        - H (qml.Hamiltonian): The Hydrogen (H2) Hamiltonian
    Returns:
        - (float): The ground state energy
        - (np.ndarray): The ground state calculated through your optimization routine
    """

    # QHACK #
    
    # Initialize thhe fevice
    num_qubits = len(H.wires)
    qubits = H.wires
    num_param_sets = (2 ** num_qubits) - 1
    dev = qml.device("default.qubit", wires=num_qubits)

    # Initialize the energy, theta and hi state
    energy = 0
    hi = np.array([1, 1, 0, 0])
    theta = np.random.uniform(low=-np.pi / 2, high=np.pi / 2, size=(1))

    # Create circuit for VQE
    def circuit(theta):
        qml.DoubleExcitation(theta[0], wires=qubits)

    # Define cost function and a function to return the ground_state
    @qml.qnode(dev)
    def cost_fn(theta):
        qml.BasisState(hi, wires=qubits)
        circuit(theta)
        return qml.expval(H)
    @qml.qnode(dev)
    def get_ground_state(theta):
        qml.BasisState(hi, wires=qubits)
        circuit(theta)
        return qml.state()

    # Define the optimizer, parameters of the VQE and lists to keep track of parameter and results at every iteration
    opt = qml.optimize.AdamOptimizer(0.1)
    max_iterations = 200
    conv_tol = 1e-06
    energy = [cost_fn(theta)]
    theta_col = [theta]
    states = [get_ground_state(theta)]

    # Perform VQE to find the angle theta
    for n in range(max_iterations):
        theta, prev_energy = opt.step_and_cost(cost_fn, theta)

        energy.append(cost_fn(theta))
        theta_col.append(theta)
        states.append(get_ground_state(theta))

        conv = np.abs(energy[-1] - prev_energy)

        if n % 2 == 0:
            print(f"Step = {n},  Energy = {energy[-1]:.8f} HH")

        if conv <= conv_tol:
            break
    print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} HH")
    print("\n" f"Optimal value of the circuit parameters = {theta_col[-1]}")
    print("\n" f"Ground-state state vector = {states[-1]}")

    # Return the ground state energy and the ground state
    return energy[-1], states[-1]
    # QHACK #


In [3]:
def create_H1(ground_state, beta, H):
    """Create the H1 matrix, then use `qml.Hermitian(matrix)` to return an observable-form of H1.
    Args:
        - ground_state (np.ndarray): from the ground state VQE calculation
        - beta (float): the prefactor for the ground state projector term
        - H (qml.Hamiltonian): the result of hf.generate_hamiltonian(mol)()
    Returns:
        - (qml.Observable): The result of qml.Hermitian(H1_matrix)
    """

    # QHACK #
    # Define function to decompose hamiltonian into observables and coefficients
    from functools import reduce
    from itertools import product
    from operator import matmul
    def decompose_hamiltonian(H):
        N = int(np.log2(len(H)))
        paulis = [qml.Identity, qml.PauliX, qml.PauliY, qml.PauliZ]

        obs = []
        coeffs = []

        for term in product(paulis, repeat=N):
            matrices = [i._matrix() for i in term]
            coeff = np.trace(reduce(np.kron, matrices) @ H) / (2 ** N)

            if not np.allclose(coeff, 0):
                coeffs.append(coeff)

                if not all(t is qml.Identity for t in term):
                    obs.append(reduce(matmul, [t(i) for i, t in enumerate(term) if t is not qml.Identity]))
                else:
                    obs.append(reduce(matmul, [t(i) for i, t in enumerate(term)]))

        return coeffs, obs

    # Get new coefficients and new observables to add to H
    gs = ground_state.reshape(-1, 1)
    new_coefficients, new_observables = decompose_hamiltonian(beta * (gs @ gs.T))

    # Add new observables and new coefficients
    new_coefficients = np.tensor(new_coefficients)
    new_coefficients = np.concatenate((new_coefficients, H.terms[0]))
    new_observables += H.terms[1]
    
    # Create new H1 hamiltonian out of new lists of coefficients and observables
    H1 = qml.Hamiltonian(new_coefficients, new_observables)
    return H1
    # QHACK #

In [4]:
def excited_state_VQE(H1):
    """Perform VQE using the "excited state" Hamiltonian.
    Args:
        - H1 (qml.Observable): result of create_H1
    Returns:
        - (float): The excited state energy
    """

    # QHACK #
    # Initialize thhe fevice
    num_qubits = len(H1.wires)
    qubits = H1.wires
    num_param_sets = (2 ** num_qubits) - 1
    dev = qml.device("default.qubit", wires=num_qubits)

    # Initialize the energy, theta and hi state
    energy = 0
    hi = np.array([1, 1, 0, 0])
    theta = np.random.uniform(low=-np.pi / 2, high=np.pi / 2, size=(1))

    # Create circuit for VQE
    def circuit(theta):
        qml.SingleExcitation(theta[0], wires=[1, 2])

    # Define cost function and a function to return the ground_state
    @qml.qnode(dev)
    def cost_fn(theta):
        qml.BasisState(hi, wires=qubits)
        circuit(theta)
        return qml.expval(H1)
    @qml.qnode(dev)
    def get_ground_state(theta):
        qml.BasisState(hi, wires=qubits)
        circuit(theta)
        return qml.state()

    # Define the optimizer, parameters of the VQE and lists to keep track of parameter and results at every iteration
    opt = qml.optimize.MomentumOptimizer()
    max_iterations = 300
    conv_tol = 1e-05
    energy = [cost_fn(theta)]
    theta_col = [theta]
    states = [get_ground_state(theta)]

    # Perform VQE to find the angle theta
    for n in range(max_iterations):
        theta, prev_energy = opt.step_and_cost(cost_fn, theta)

        energy.append(cost_fn(theta))
        theta_col.append(theta)
        states.append(get_ground_state(theta))

        conv = np.abs(energy[-1] - prev_energy)

        if n % 2 == 0:
            print(f"Step = {n},  Energy = {energy[-1]:.8f} HH")

        if conv <= conv_tol:
            break
    print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} HH")
    print("\n" f"Optimal value of the circuit parameters = {theta_col[-1]}")
    print("\n" f"Ground-state state vector = {states[-1]}")

    # Return the ground state energy and the ground state
    return energy[-1]
    # QHACK #

In [5]:
coord = 0.6614
symbols = ["H", "H"]
geometry = np.array([[0.0, 0.0, -coord], [0.0, 0.0, coord]], requires_grad=False)
mol = hf.Molecule(symbols, geometry)

H = hf.generate_hamiltonian(mol)()
E0, ground_state = ground_state_VQE(H)

beta = 15.0 + E0
H1 = create_H1(ground_state, beta, H)
E1 = excited_state_VQE(H1)

print(f"Ground state energy E_0 = {E0.item()}")
print(f"First excited state energy E_0 = {E1.item()}")

/opt/conda/envs/pennylane/lib/python3.9/site-packages/pennylane/operation.py:1830: UserWarning: Tensor object acts on overlapping wires; in some PennyLane functions this will lead to undefined behaviour
  warnings.warn(
/opt/conda/envs/pennylane/lib/python3.9/site-packages/pennylane/operation.py:1993: UserWarning: Tensor object acts on overlapping wires; in some PennyLane functions this will lead to undefined behaviour
  warnings.warn(
/opt/conda/envs/pennylane/lib/python3.9/site-packages/pennylane/__init__.py:350: UserWarning: The qml.grouping module is deprecated, please use qml.pauli instead.
  warnings.warn(


Step = 0,  Energy = -0.28886989 HH
Step = 2,  Energy = -0.45940995 HH
Step = 4,  Energy = -0.62239941 HH
Step = 6,  Energy = -0.77081794 HH
Step = 8,  Energy = -0.89823909 HH
Step = 10,  Energy = -0.99958845 HH
Step = 12,  Energy = -1.07202666 HH
Step = 14,  Energy = -1.11574902 HH
Step = 16,  Energy = -1.13434801 HH
Step = 18,  Energy = -1.13435479 HH
Step = 20,  Energy = -1.12384373 HH
Step = 22,  Energy = -1.11052045 HH
Step = 24,  Energy = -1.10009150 HH
Step = 26,  Energy = -1.09552117 HH
Step = 28,  Energy = -1.09721444 HH
Step = 30,  Energy = -1.10376156 HH
Step = 32,  Energy = -1.11284552 HH
Step = 34,  Energy = -1.12206150 HH
Step = 36,  Energy = -1.12952788 HH
Step = 38,  Energy = -1.13423297 HH
Step = 40,  Energy = -1.13609807 HH
Step = 42,  Energy = -1.13578524 HH
Step = 44,  Energy = -1.13433832 HH
Step = 46,  Energy = -1.13278972 HH
Step = 48,  Energy = -1.13186116 HH
Step = 50,  Energy = -1.13183320 HH
Step = 52,  Energy = -1.13258530 HH
Step = 54,  Energy = -1.13375126 

AttributeError: type object 'Identity' has no attribute '_matrix'